The trained model is present with the name : CascadeRCNN_X101 VinBigData 20Ep MMDET in the datasets. 

In [1]:
import os
import cv2
import random


In [2]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_19:24:38_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
%%time

print("this will take around 10 mins")
# install dependencies: (use cu101 because colab has CUDA 10.1)
# !pip install -U torch==1.7.0+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full


this will take around 10 mins
     |████████████████████████████████| 231 kB 865 kB/s 
     |████████████████████████████████| 185 kB 3.3 MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.2.7-cp37-cp37m-linux_x86_64.whl size=20707359 sha256=9594dc1f21fb1aa6931ff884c2b473a450603ac7a9e7586fe585dc4f0f11ae9c
  Stored in directory: /root/.cache/pip/wheels/3f/3d/5e/dbc82b2ea5f1e14d4fe044aea9aeb02a8a9402c051529659c2
Successfully built mmcv-full
CPU times: user 7.87 s, sys: 1.22 s, total: 9.1 s
Wall time: 8min 33s


In [4]:
!rm -rf mmdetection
!git clone --branch v2.7.0 https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

Cloning into 'mmdetection'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 16664 (delta 28), reused 14 (delta 4), pack-reused 16529
Receiving objects: 100% (16664/16664), 20.33 MiB | 22.48 MiB/s, done.
Resolving deltas: 100% (11475/11475), done.
Note: checking out '3e902c3afc62693a71d672edab9b22e35f7d4776'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

/kaggle/working/mmdetection
Obtaining file:///kaggle/working/mmdetection
  Created wheel for mmpycocotools: filename=mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl size=272902 s

In [5]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.7.0 True
2.7.0
10.2
GCC 7.5


# CascadeRNN101X Pretrained

In [6]:
!mkdir checkpoints
# !wget -c http://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth \
#       -O checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth


!wget -c https://s3.ap-northeast-2.amazonaws.com/open-mmlab/mmdetection/models/cascade_rcnn_x101_32x4d_fpn_1x_20190501-af628be5.pth \
      -O checkpoints/cascade_rcnn_x101_32x4d_fpn_1x_20190501-af628be5.pth


--2021-03-31 04:14:31--  https://s3.ap-northeast-2.amazonaws.com/open-mmlab/mmdetection/models/cascade_rcnn_x101_32x4d_fpn_1x_20190501-af628be5.pth
Resolving s3.ap-northeast-2.amazonaws.com (s3.ap-northeast-2.amazonaws.com)... 52.219.58.117
Connecting to s3.ap-northeast-2.amazonaws.com (s3.ap-northeast-2.amazonaws.com)|52.219.58.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 352728209 (336M) [application/x-www-form-urlencoded]
Saving to: ‘checkpoints/cascade_rcnn_x101_32x4d_fpn_1x_20190501-af628be5.pth’

checkpoints/cascade 100%[===================>] 336.39M  11.5MB/s    in 31s     

2021-03-31 04:15:03 (10.8 MB/s) - ‘checkpoints/cascade_rcnn_x101_32x4d_fpn_1x_20190501-af628be5.pth’ saved [352728209/352728209]



In [7]:
test_anno = "../../input/vinbigdata-1024-image-dataset/vinbigdata/test"

ids = os.listdir(test_anno)

In [8]:
len(ids)

3000

# Preparing Test Annotation file

In [9]:
img_infos = []
for i, _id in enumerate(ids):
    if '.png' in _id:
        img_infos.append({
                    "license": 0,
                    "url": 'null',
                    "file_name": _id,
                    "height": 1024,
                    "width": 1024,
                    "date_captured": 'null',
                    "id": _id
                })

In [10]:
img_infos[0]

{'license': 0,
 'url': 'null',
 'file_name': 'b3f67ac077531f44dd06275af31edbd9.png',
 'height': 1024,
 'width': 1024,
 'date_captured': 'null',
 'id': 'b3f67ac077531f44dd06275af31edbd9.png'}

In [11]:
import json
val_anno = '../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_annotations.json'

with open(val_anno) as f:
    dd = json.load(f)

dd.keys()
dd['annotations']=[]
dd['images']
dd['images'] = img_infos
with open('./test_ann.json', 'w') as outfile:
    json.dump(dd, outfile)

In [12]:
val_ids = os.listdir('../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_images')

In [13]:
_classes = ("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")

# Custom Config Options

In [14]:
_cfg_options = {"dataset_type" : 'CocoDataset',
"classes" : '''("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")''',
"data.train.img_prefix" : '../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/',
"data.train.classes" : '''("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")''',
"data.train.ann_file" : '../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/train_annotations.json',
"data.train.type" : 'CocoDataset',
"data.val.img_prefix" : '../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/',
"data.val.classes" : '''("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")''',
"data.val.ann_file" : '../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_annotations.json',
"data.val.type" : 'CocoDataset',
"data.test.img_prefix" : '../../input/vinbigdata-1024-image-dataset/vinbigdata/test/',
"data.test.classes" : '''("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")''',
"data.test.ann_file" : './test_ann.json',
"data.test.type":'CocoDataset',
"data.train.type" : 'CocoDataset',
"data.val.type" : 'CocoDataset',
"data.test.type" : 'CocoDataset',
"log_config.interval" : 10,
"evaluation.metric" : 'bbox',
"load_from" : './checkpoints/cascade_rcnn_x101_32x4d_fpn_1x_20190501-af628be5.pth',
"work_dir" : "../vinbig_output",
"total_epochs" : '21'}


# "model.roi_head.bbox_head[0].num_classes" : '14',
# "model.roi_head.bbox_head[1].num_classes" : '14',
# "model.roi_head.bbox_head[2].num_classes" : '14',

cfg_op = ""
for k, v in _cfg_options.items():
    cfg_op+=f"{k}='{v}' "
print(cfg_op)


dataset_type='CocoDataset' classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.train.img_prefix='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/' data.train.classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.train.ann_file='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/train_annotations.json' data.train.type='CocoDataset' data.val.img_prefix='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/' data.val.classes='("Aorti

# Loading the 20ep trained model

In [15]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmcv import Config

# Choose to use a config and initialize the detector
# config = 'configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py'
config = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_x101_32x4d_fpn_1x_coco.py')
config.model.roi_head.bbox_head[0].num_classes = 14
config.model.roi_head.bbox_head[1].num_classes = 14
config.model.roi_head.bbox_head[2].num_classes = 14
checkpoint = '../../input/cascadercnnx-vinbigdata-20ep-1024/epoch_20.pth'
# initialize the detector

model = init_detector(config, checkpoint, device='cuda:0', cfg_options=_cfg_options)
model.CLASSES = _classes


Use load_from_local loader


/kaggle/working/mmdetection/mmdet/apis/inference.py:46: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


# Option 1 for inference - Using only 10 images for demo


In [16]:
_id = random.randint(1,1098)
# Use the detector to do inference
# img = f'../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_images/{val_ids[_id]}'
from tqdm import tqdm
test_img_ids = os.listdir("../../input/vinbigdata-1024-image-dataset/vinbigdata/test")
result = {}
for _id in tqdm(test_img_ids, total=len(test_img_ids)):
    img_path = "../../input/vinbigdata-1024-image-dataset/vinbigdata/test/" + f"{_id}"
    pred = inference_detector(model, img_path)
    result[_id] = pred
# img = "../../input/vinbigdata-1024-image-dataset/vinbigdata/test/002a34c58c5b758217ed1f584ccbcfe9.png"
# result = inference_detector(model, img)
# show_result_pyplot(model, img, result, score_thr=0.3)

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 3000/3000 [08:43<00:00,  5.74it/s]


# Option 2 for inference

In [17]:
# !python tools/test.py ./configs/cascade_rcnn/cascade_rcnn_x101_32x4d_fpn_1x_coco.py ../../input/cascadercnnx-vinbigdata-20ep-1024/epoch_20.pth --cfg-options dataset_type='CocoDataset' classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.train.img_prefix='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/' data.train.classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.train.ann_file='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/train_annotations.json' data.train.type='CocoDataset' data.val.img_prefix='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/' data.val.classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.val.ann_file='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_annotations.json' data.val.type='CocoDataset' data.test.img_prefix='../../input/vinbigdata-1024-image-dataset/vinbigdata/test/' data.test.classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.test.ann_file='./test_ann.json' data.test.type='CocoDataset' log_config.interval='10' evaluation.metric='bbox' load_from='./checkpoints/cascade_rcnn_x101_32x4d_fpn_1x_20190501-af628be5.pth' work_dir='../vinbig_output' total_epochs='21' --out preds_cascadex.pkl

In [18]:
os.listdir("./")

['mmdet',
 'README.md',
 'resources',
 'setup.cfg',
 '.git',
 'tests',
 '.gitignore',
 'requirements',
 '.github',
 'configs',
 'setup.py',
 'docs',
 '.dev_scripts',
 'test_ann.json',
 'pytest.ini',
 '.readthedocs.yml',
 'requirements.txt',
 'checkpoints',
 '.pre-commit-config.yaml',
 'LICENSE',
 'tools',
 'docker',
 'mmdet.egg-info',
 'demo']

In [19]:
# import pickle

# with open('./preds_cascadex.pkl', 'rb') as f:
#     data = pickle.load(f)

In [20]:

import json
with open('./test_ann.json', 'rb') as f:
    ann = json.load(f)

In [21]:
# file_ids = [file_name.get('file_name').split('.png')[0] for file_name in ann.get('images')]

In [22]:
!pip install ensemble_boxes

In [23]:
import pandas as pd
test_df = pd.read_csv('../../input/vinbigdata-original-image-dataset/vinbigdata/test.csv')

In [24]:
from ensemble_boxes import *

In [25]:
# ann_with_pred = zip(file_ids, data)
submission_vals = []
# for _id, preds in ann_with_pred


# this method is for option1
for _id, preds in result.items():
    boxes = []
    scores = []
    labels = []
    _id = _id.split('.png')[0]
    width = test_df[test_df.image_id==_id]['width'].iloc[0]
    height = test_df[test_df.image_id==_id]['height'].iloc[0]  
    for i, pred in enumerate(preds):
        if len(pred):
            for p in pred:
                box = p[:4]/1024
                boxes.append(box)
                score = p[4].astype(float)
                scores.append(score)
                labels.append(i)
    boxes, scores, labels = weighted_boxes_fusion([boxes], [scores], [labels], iou_thr=0.4, skip_box_thr=0.4)
    boxes[:, 0] = boxes[:, 0]*height
    boxes[:, 2] = boxes[:, 2]*height
    boxes[:, 1] = boxes[:, 1]*width
    boxes[:, 3] = boxes[:, 3]*width
    
    scaled_boxes = boxes.astype(int)
    labels = labels.astype(int)
    _id_preds = []
    if len(scaled_boxes):
        for i in range(len(scaled_boxes)):
            _id_preds.append(str(labels[i]))
            _id_preds.append(str(scores[i].round(2)))
            _id_preds.append(str(scaled_boxes[i][0]))
            _id_preds.append(str(scaled_boxes[i][1]))
            _id_preds.append(str(scaled_boxes[i][2]))
            _id_preds.append(str(scaled_boxes[i][3]))
        pred_str = " ".join(_id_preds)
    else:
        pred_str = '14 1 0 0 1 1'
    submission_vals.append([_id, pred_str])



In [26]:
df = pd.DataFrame(submission_vals, columns = ['image_id','PredictionString'])

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
result['e94fde220360e4b769921e16059cc6af.png']

[array([[5.4652924e+02, 2.5355429e+02, 6.4508380e+02, 3.7259195e+02,
         9.8796332e-01],
        [5.6558691e+02, 2.5726105e+02, 6.4021991e+02, 3.3424710e+02,
         7.7831805e-02]], dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([[4.3461157e+02, 5.7348163e+02, 8.2128790e+02, 7.3016461e+02,
         7.2105485e-01]], dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([[1.1172021e+02, 7.9010846e+02, 1.4882402e+02, 8.4127545e+02,
         2.1870145e-01]], dtype=float32),
 array([[1.1283211e+02, 7.9126276e+02, 1.4906384e+02, 8.4167542e+02,
         2.9732403e-01]], dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32)]

In [28]:


result['b3f67ac077531f44dd06275af31edbd9.png']



[array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([[6.55316956e+02, 1.03516174e+02, 7.81057434e+02, 1.33637192e+02,
         6.88432038e-01],
        [3.56639923e+02, 1.13535706e+02, 4.47455872e+02, 1.29955215e+02,
         1.39796942e-01],
        [6.60883423e+02, 1.05171867e+02, 7.66113586e+02, 1.21728271e+02,
         8.53050575e-02]], dtype=float32),
 array([], shape=(0, 5), dtype=float32),
 array([[7.2755298e+02, 6.3622113e+02, 7.9987531e+02, 6.7070123e+02,
         4.0024024e-01],
        [7.2495068e+02, 5.7403680e+02, 8.0462994e+02, 6.7946912e+02,
         7.2166771e-02],
      

In [29]:
df.head()

,image_id,PredictionString
0,b3f67ac077531f44dd06275af31edbd9,11 0.69 1965 310 2343 400 13 0.4 2182 1908 239...
1,e94fde220360e4b769921e16059cc6af,0 0.99 1296 493 1530 725 3 0.72 1031 1116 1948...
2,d2993271d0a49d1d821db1174861629a,0 0.99 1349 671 1685 939 4 0.49 456 715 1004 9...
3,99732e844196448b90e50973d6ba5224,3 0.99 1249 1399 2294 1799 0 0.99 1617 684 188...
4,3c6f70f56258cad13d74e7f17da1c7c5,3 0.99 1164 1292 2361 1609 0 0.98 1555 712 189...


In [30]:
len(df)

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


3000

In [31]:
df[df.PredictionString=="14 1 0 0 1 1"].count()
                                             

image_id            424
PredictionString    424
dtype: int64

In [32]:
df.tail(10)

,image_id,PredictionString
2990,269af0bfa0e3246aa7b7330a3c510030,3 0.94 875 1144 2168 1649 0 0.79 1439 569 1770...
2991,e06e6af1b1ae040355e80dc109d8406d,3 0.97 1218 1539 2211 1945 0 0.96 1522 779 179...
2992,80f3da71b6af9134388dee22f0d386d9,0 0.73 1039 740 1260 1051
2993,0403dda5a9bf46457517b604869d530d,14 1 0 0 1 1
2994,edb0b27fb3602ffb157e5a022e6b5a70,14 1 0 0 1 1
2995,e71ff456b51d655f9b70ecbc6fe2572f,8 0.85 374 755 437 812 3 0.81 902 997 2444 152...
2996,9c39e251d5c438f0320cbb39fa33b351,0 0.88 1372 609 1601 805
2997,71c75db6da20e3320fb90ca9b018ab31,3 0.83 975 1145 1932 1487
2998,2e2f4e832dc3bd12ddd8d405cbf7fdf5,0 0.98 1221 510 1486 737 3 0.86 864 1027 1860 ...
2999,69aae4076223158f2e0cfe8f6f74ce68,3 0.44 1062 995 2025 1284


In [33]:
df.iloc[2222]['PredictionString']

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'0 0.98 1343 573 1650 904'

In [34]:
df.to_csv('submission.csv', header=True, index=False)